# Wedding Seating Optimization

This work was conducted by Group G:

- Daniel Caridade (20211588)
- Gonçalo Teles (20211684)
- Gonçalo Peres (20211625)
- Guilherme Godinho (20211552)

# 1. Library Importation

__`Step 1`__ Import the required libraries.

In [ ]:
import pandas as pd
import random
import numpy as np
import copy
import math

# 2. Data Integration

__`Step 2`__ Importing the dataset into the file.

In [ ]:
from utils.parser import load_relationship_matrix

# Load the data
relationships = load_relationship_matrix()
print(relationships.head())

       1     2     3     4    5    6    7    8    9    10  ...   55   56  57  \
idx                                                        ...                 
1       0  5000     0     0  700  700    0    0    0    0  ...  100  100   0   
2    5000     0   700   700    0    0  300  300  500  500  ...  100  100   0   
3       0   700     0  2000    0    0    0    0  300  300  ...    0    0   0   
4       0   700  2000     0    0    0  900  400  300  300  ...    0    0   0   
5     700     0     0     0    0    0    0    0    0    0  ...    0    0   0   

      58   59   60   61  62  63  64  
idx                                  
1      0  100  100  100   0   0   0  
2    100    0    0    0   0   0   0  
3      0    0    0    0   0   0   0  
4      0    0    0    0   0   0   0  
5      0    0    0    0   0   0   0  

[5 rows x 64 columns]


# 3. Helper functions

__`Step 3`__

In [ ]:
from utils.WeddingSeatingHelper import WeddingSeatingHelper # adjust import path accordingly

# Suppose relationships is loaded before, e.g. from your parser:
helper = WeddingSeatingHelper(relationships)

solution = helper.generate_solution()
fitness_score = helper.fitness(solution)
neighbors = helper.get_neighbours(solution)

## Class for HillCLlimbing and testing

In [18]:
class HillClimbingOptimizer:
    import numpy as np
    def __init__(self, helper):
        """
        :param helper: An instance of WeddingSeatingHelper or similar class providing necessary methods
        """
        self.helper = helper

    def run(self, verbose=False):
        """
        Run the hill climbing optimization.
        
        :param verbose: If True, prints fitness progress.
        :return: tuple(best_fitness, best_solution)
        """
        current_sol = self.helper.generate_solution()
        neighbours = self.helper.get_neighbours(current_sol)
        neighbour_best_fit = max([self.helper.fitness(neighbour) for neighbour in neighbours])

        while self.helper.fitness(current_sol) < neighbour_best_fit:
            current_sol = neighbours[np.argmax([self.helper.fitness(neighbour) for neighbour in neighbours])]
            neighbours = self.helper.get_neighbours(current_sol)
            neighbour_best_fit = max([self.helper.fitness(neighbour) for neighbour in neighbours])

            if verbose:
                print(f"Current fitness: {self.helper.fitness(current_sol)}")

        return self.helper.fitness(current_sol), current_sol

In [ ]:
from optimizers.HillClimbing import HillClimbingOptimizer

helper = WeddingSeatingHelper(relationships)
hc = HillClimbingOptimizer(helper)

best_fit, best_sol = hc.run(verbose=True)
print("Best fitness:", best_fit)

Current fitness: 19900
Current fitness: 25100
Current fitness: 30800
Current fitness: 34900
Current fitness: 38900
Current fitness: 42200
Current fitness: 45900
Current fitness: 49900
Current fitness: 52600
Current fitness: 55000
Current fitness: 57200
Current fitness: 59500
Current fitness: 61700
Current fitness: 63700
Current fitness: 65600
Current fitness: 67200
Current fitness: 68700
Current fitness: 70300
Current fitness: 72200
Current fitness: 73900
Current fitness: 75000
Current fitness: 77000
Current fitness: 77400
Current fitness: 77600
Current fitness: 77700
Best fitness: 77700


In [ ]:
results_hc = []

for i in range(30):
    best_fit, best_sol = hc.run()
    results_hc.append(best_fit)
    print(f"Run {i+1}: Best fitness = {best_fit}")

# Optionally, summarize results_hc
print(f"\nAverage fitness over 30 runs: {sum(results_hc)/len(results_hc):.2f}")
print(f"Best fitness found: {max(results_hc)}")
print(f"Worst fitness found: {min(results_hc)}")

## Simulated Annealing in classes 

In [20]:
import numpy as np
import random
import math

class SimulatedAnnealingOptimizer:
    def __init__(self, helper, L=200, k=1.1, c=1000000, stop=5, seed=42):
        """
        :param helper: An instance of WeddingSeatingHelper
        :param L: Number of temperature iterations
        :param k: Cooling rate divisor
        :param c: Initial temperature
        :param stop: Early stop count for no improvement
        :param seed: Random seed for reproducibility
        """
        self.helper = helper
        self.L = L
        self.k = k
        self.c = c
        self.stop = stop
        random.seed(seed)
        np.random.seed(seed)

    def run(self, verbose=False):
        current_sol = self.helper.generate_solution()
        count = 0
        temperature = self.c

        for _ in range(self.L):
            current_fit = self.helper.fitness(current_sol)
            neighbours = self.helper.get_neighbours(current_sol)
            neighbours_fitness = [(n, self.helper.fitness(n)) for n in neighbours]
            neighbours_fitness.sort(key=self.helper.sorter, reverse=True)

            for neighbour, neighbour_fit in neighbours_fitness:
                delta = abs(current_fit - neighbour_fit)
                if random.random() < math.exp(delta / temperature):
                    current_sol = neighbour
                    break

            if current_fit >= self.helper.fitness(current_sol):
                count += 1
            else:
                count = 0  # Reset if improvement

            if count == self.stop:
                break

            if verbose:
                print(f"Current fitness: {self.helper.fitness(current_sol)}")

            temperature /= self.k

        return self.helper.fitness(current_sol), current_sol

In [ ]:
from code.optimizers.SimulatedAnnealing import SimulatedAnnealingOptimizer

# Run Simulated Annealing
sa = SimulatedAnnealingOptimizer(helper)
best_fit, best_sol = sa.run(verbose=True)
print("Best fitness:", best_fit)

Current fitness: 16300
Current fitness: 21300
Current fitness: 26300
Current fitness: 30700
Current fitness: 35100
Current fitness: 39200
Current fitness: 42500
Current fitness: 46000
Current fitness: 49000
Current fitness: 51800
Current fitness: 54700
Current fitness: 57400
Current fitness: 59800
Current fitness: 62400
Current fitness: 64500
Current fitness: 66600
Current fitness: 68600
Current fitness: 70600
Current fitness: 71700
Current fitness: 74200
Current fitness: 75700
Current fitness: 76000
Current fitness: 76100
Current fitness: 76400
Current fitness: 76500
Current fitness: 76500
Current fitness: 76500
Current fitness: 76500
Current fitness: 76500
Best fitness: 76500
